# Test ResNet Model

### Author: Josh Hudziak
***
### Date: 30/04/21
***
### Purpose:
* Test a model that has been trained on the dataset against real world footage of new locations
***
### Copyright:
    This work is licensed under the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License. To view a copy of this license, visit http://creativecommons.org/licenses/by-nc-sa/4.0/.


## Import Libraries

In [1]:
# import the necessary packages
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from imutils import paths
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import os

In [2]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model('MODELS/Gun.h5')
lb = pickle.loads(open('MODELS/lb.pickle', "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
Q = deque(maxlen = 2)

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture('../CNN UAV Project/_data/_HOUSE/centre.mov')
writer = None
(W, H) = (None, None)

# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 250x250, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (250, 250)).astype("float32")
    

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    
    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    
    label = lb.classes_[i]
    #print(label)
    
    # draw the activity on the output frame
    text = "Operation: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.25, (0, 255, 0), 5)
    
    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('Video_With_Classification', fourcc, 60,
            (W, H), True)
        
    # write the output frame to disk
    writer.write(output)
    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()


[INFO] loading model and label binarizer...
[INFO] cleaning up...
